<a href="https://colab.research.google.com/github/z-arabi/notebooks/blob/main/01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

Cloning into 'notebooks'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 283 (delta 161), reused 148 (delta 70), pack-reused 0
Receiving objects: 100% (283/283), 13.55 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (161/161), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [2]:
#hide
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.
Using transformers v4.11.3
Using datasets v1.16.1


# Hello Transformers

## The Encoder-Decoder Framework

## Attention Mechanisms

## Transfer Learning in NLP

## Hugging Face Transformers: Bridging the Gap

## A Tour of Transformer Applications

In [3]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

## pipelines
https://huggingface.co/docs/transformers/main_classes/pipelines 

### Text Classification

In [4]:
#hide_output
from transformers import pipeline

classifier = pipeline("text-classification")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [5]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)    

,label,score
0,NEGATIVE,0.901546


In [7]:
my_text = """Hi, I got a device from amazon, but I did not like its color. \
In addition to its function, this device also had a beautiful appearance. \
My only problem with this device was its color."""

my_output = classifier(my_text)
pd.DataFrame(my_output)    

,label,score
0,POSITIVE,0.993431


In [14]:
print(my_output) # the lis of dictionary
# convert the list of dictionary to the DF
pd.DataFrame(my_output) 

[{'label': 'POSITIVE', 'score': 0.993431031703949}]


,label,score
0,POSITIVE,0.993431


### Named Entity Recognition

In [6]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)    

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


In [8]:
ner_output = ner_tagger(my_text)
pd.DataFrame(ner_output) 

,entity_group,score,word,start,end
0,ORG,0.817922,amazon,24,30


### Question Answering 

In [9]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])    

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


In [11]:
my_question = "What was the customer dissatisfied with?"
q_output = reader(question=my_question, context=my_text)
pd.DataFrame([q_output])

,score,start,end,answer
0,0.249746,173,182,its color


### Summarization

In [19]:
summarizer = pipeline("summarization")

In [18]:
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs)
print('*'*10)
print(outputs[0]['summary_text'])

[{'summary_text': ' Bumblebee ordered an Optimus Prime action figure from your
online store in Germany. Unfortunately, when I opened the package, I discovered
to my horror that I had been sent an action figure of Megatron instead.'}]
**********
 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead.


In [25]:
sum_output = summarizer(my_text, clean_up_tokenization_spaces=True)
print(sum_output[0]['summary_text'])

 My only problem with this device was its color. Hi, I got a device from amazon,
but I did not like its color. In addition to its function, this device also had
a beautiful appearance. My only complaint was that the color was that of the
device.


### Translation

In [20]:
translator = pipeline("translation_en_to_de", 
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus
Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete,
entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von
Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich
hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere
einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt.
Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von
Ihnen zu hören. Aufrichtig, Bumblebee.


In [37]:
# https://huggingface.co/languages
# ValueError: The task does not provide any default models for options ('en', 'fa')

my_translator = pipeline("translation_en_to_it",model="Helsinki-NLP/opus-mt-en-it")
translate_output = my_translator(my_text, clean_up_tokenization_spaces=True, min_length=100)
print(translate_output[0]['translation_text'])

Ciao, ho ottenuto un dispositivo da amazon, ma non mi è piaciuto il suo colore.
Oltre alla sua funzione, questo dispositivo ha anche avuto un aspetto
bellissimo. Il mio unico problema con questo dispositivo era il suo colore.
@info: whatsthis. textbox text text text text text this text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text text text text text text text
text text text text text text text text text text


### Text Generation

In [21]:
#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [22]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. The order was
completely mislabeled, which is very common in our online store, but I can
appreciate it because it was my understanding from this site and our customer
service of the previous day that your order was not made correct in our mind and
that we are in a process of resolving this matter. We can assure you that your
order


In [39]:
my_response = "Dear Customer, we will take care of your problem ."
prompt = my_text + "\n\nCustomer service response:\n" + my_response
gen_output = generator(prompt, max_length=200)
print(gen_output[0]['generated_text'])

Hi, I got a device from amazon, but I did not like its color. In addition to its
function, this device also had a beautiful appearance. My only problem with this
device was its color.

Customer service response:
Dear Customer, we will take care of your problem . Please consider purchasing a
new one as soon as possible, or we can make a replacement for you.

Customer service response:

Excellent device. I'm impressed with the color. The overall quality was very
good. The color was very bright and easy to read. The only problems I
encountered were the white backlight (the black backlight), which the phone
looked great.

Customer service response:

Not bad. The backlight is very bright and very bright.

Customer service response:

This should have been a nice feature! Would not normally care on something like
this. However, I wish this was a separate product rather than a separate
product, as I


## The Hugging Face Ecosystem

### The Hugging Face Hub

### Hugging Face Tokenizers

### Hugging Face Datasets

### Hugging Face Accelerate

## Main Challenges with Transformers

## Conclusion